# Tarea de la Semana 2:<br>Un Ejemplo del Ciclo de Vida de la Ingeniería de Datos


En este laboratorio, configurarás y ejecutarás un ejemplo de canalización de datos que muestra todas las etapas del ciclo de vida de la ingeniería de datos. Tu sistema fuente será una base de datos relacional instanciada como base de datos MySQL en Amazon RDS (Servicio de Base de Datos Relacional). Primero explorarás la base de datos fuente con un conjunto de datos de muestra, luego usarás AWS Glue para extraer, transformar y cargar (ETL) los datos en tu canalización de datos, almacenándolos en el almacenamiento de objetos de AWS Amazon S3. Finalmente, consultarás los datos almacenados utilizando Amazon Athena para construir un panel de visualización de datos en Jupyter Lab. Para definir y configurar los componentes de este ejemplo de canalización de datos, utilizarás Terraform como servicio de Infraestructura como Código (IaC).


# Tabla de Contenidos
- [ 1 - Introducción](#1)
- [ 2 - Explorando el Sistema Fuente](#2)
- [ 3 - Arquitectura del Pipeline de Datos](#3)
- [ 4 - Extracción, Transformación y Carga de Datos](#4)
- [ 5 - Datos e Visualizaciones](#5)


<div id='1'/>

## 1 - Introducción


Suponga que trabaja como Ingeniero de Datos en un minorista de modelos a escala de autos clásicos y otros medios de transporte. El minorista almacena sus compras históricas y la información de sus clientes en una base de datos relacional que consta de las siguientes tablas: clientes, productos, líneas de productos, pedidos, detalles de pedidos, pagos, empleados, oficinas. En este laboratorio, utilizará un ejemplo de dicha base de datos: [Base de datos de muestra de MySQL](https://www.mysqltutorial.org/mysql-sample-database.aspx)

El analista de datos del equipo de marketing está interesado en analizar las compras históricas para comprender, por ejemplo, qué línea de productos es más exitosa y cómo se distribuyen las ventas en los diferentes países. Si bien el analista de datos puede consultar directamente los datos de la base de datos relacional, es posible que necesite escribir consultas complejas que podrían tardar mucho tiempo en recuperar los datos requeridos. Realizar consultas analíticas en bases de datos de aplicaciones de producción generalmente es una mala idea, porque podría afectar el rendimiento de esa base de datos. Su trabajo como Ingeniero de Datos es construir un canal de datos que transforme los datos en una forma más fácil de entender y más rápida de consultar, y servirlo al analista de datos para que pueda centrarse únicamente en los esfuerzos analíticos.

En este laboratorio, explorará un ejemplo de extremo a extremo que muestra la implementación de las etapas del ciclo de vida de la ingeniería de datos en AWS, y se familiarizará con el entorno del laboratorio.

¡Comencemos primero explorando el sistema fuente!


<div id='2'/>

## 2 - Explorando el Sistema Fuente


Como Ingeniero de Datos, generalmente no mantienes ni creas los sistemas fuente porque están fuera de tu control. Tu trabajo es construir el pipeline de datos que ingiere y combina datos de los sistemas fuente. En este laboratorio, se te proporciona una base de datos MySQL que representa el sistema fuente con el que interactuarás. La base de datos proporcionada contiene las tablas que enumeramos en la sección anterior, y se instancia utilizando Amazon RDS, que es un servicio gestionado que te permite ejecutar bases de datos relacionales en la nube. Antes de ejecutar el pipeline de datos, explorarás rápidamente esta base de datos.


2.1. Para conectarse a la instancia de Amazon RDS MySQL, necesitas obtener su punto de conexión. 
Un punto de conexión es la URL del punto de entrada para el servicio web de AWS. 
Obtén el punto de conexión de la instancia de la base de datos con el siguiente comando 
(reemplaza `<MySQL-DB-name>` con `de-c1w2-rds`).

Ejecuta el comando en la terminal de VSCode. Si es necesario, abre 
una nueva terminal seleccionando Terminal > Nueva terminal en el menú.

``` bash
aws rds describe-db-instances --db-instance-identifier <MySQL-DB-name> --output text --query "DBInstances[].Endpoint.Address"
```


2.2. Ahora conectese a la base de datos ejecutando el siguiente comando, reemplazando
- `<MySQLEndpoint>` con la salida del paso anterior,

- `<DatabaseUserName>` con `admin` ,

- `<Password>` con `adminpwrd` :

``` bash
mysql --host=<MySQLEndpoint> --user=<DatabaseUserName> --password=<Password> --port=3306
```


2.3. Después de haber establecido la conexión con la instancia de RDS, puedes verificar la base de datos etiquetada como `classicmodels` y verificar las tablas que contiene ejecutando los siguientes comandos:

``` bash
use classicmodels;
show tables;
```

Deberías obtener una salida similar a esta:

![show_tables](./images/show_tables.png)

El script utilizado para poblar la base de datos se puede encontrar en *data/mysqlsampledatabase.sql*. Revisa este archivo e intenta identificar algunos de los comandos. No te preocupes si no entiendes todas las declaraciones; obtendrás una mejor comprensión de este lenguaje en futuros laboratorios. Como se mencionó anteriormente, la base de datos ya está poblada para ti, por lo que no es necesario que modifiques o ejecutes este archivo.


2.4. Ingresa `exit` en la terminal para cerrar la conexión con la base de datos.

``` bash
exit
```


2.5. También puedes verificar el RDS en la consola de AWS. Para acceder a la consola, ejecuta el siguiente comando en la terminal.

```bash
cat ../.aws/aws_console_url
```
Abre el enlace en una nueva ventana del navegador. Encuentra **RDS**, haz clic en **Bases de datos** en el panel izquierdo y verifica que la base de datos `de-c1w2-rds` esté disponible.

*Nota*: Por razones de seguridad, la URL para acceder a la consola de AWS caducará cada 15 minutos, 
pero cualquier recurso de AWS que hayas creado seguirá estando disponible durante el período de 2 horas. 
Si necesitas acceder a la consola después de 15 minutos, vuelve a ejecutar el comando para obtener un nuevo enlace activo.

*Nota*: Si ves la ventana como en la siguiente captura de pantalla, haz clic en el enlace **cerrar sesión**, 
cierra la ventana y vuelve a hacer clic en el enlace de la consola.

![AWSLogout](images/AWSLogout.png)


<div id='3'/>

## 3 - Arquitectura del Pipeline de Datos


Aquí está la arquitectura propuesta del pipeline de datos. Esta arquitectura puede servir como una solución al problema presentado en este laboratorio. Extrae los datos de la instancia de base de datos RDS MySQL y utiliza AWS Glue para transformar y almacenar los datos en un bucket de S3 (instancia de almacenamiento de objetos), haciéndolos consultables para el analista de datos a través de Amazon Athena.

![etl_diagram](images/ETL.drawio.png)

Aquí hay una breve descripción de los componentes:

- **Base de datos de origen:** Ya interactuaste con la base de datos de origen en la sección anterior. Esta es una base de datos relacional que alberga tablas estructuradas y normalizadas. Más específicamente, representa un sistema de Procesamiento de Transacciones en Línea (OLTP) que almacena datos transaccionales, donde una transacción en este laboratorio representa una orden de venta realizada por un cliente.

- **Extracción, Transformación y Carga (ETL):** El segundo componente es [AWS Glue](https://aws.amazon.com/en/glue/).

  - **Extracción:** Este paso implica extraer datos de la base de datos OLTP. Se utiliza un trabajo de AWS Glue para conectarse a la base de datos RDS y recuperar datos.

  - **Transformación:** Después de extraer los datos, Glue realiza la transformación de datos. Puedes especificar a Glue qué tipo de transformación te gustaría realizar en los datos extraídos. Para este laboratorio, la transformación consiste en modelar los datos en un esquema estrella, que difiere del de la base de datos de origen. El esquema estrella es una forma que hace que los datos sean más legibles y más fáciles de usar para el analista de datos, y simplifica la escritura de consultas para extraer los datos necesarios para su análisis. Convertir los datos en un esquema estrella puede incluir tareas como desnormalización, agregaciones y cualquier limpieza o enriquecimiento de datos necesario.

  - **Carga:** Este paso implica almacenar los datos transformados en un sistema de almacenamiento. El sistema de almacenamiento elegido en este laboratorio es la solución de almacenamiento de objetos de AWS: [Amazon S3](https://aws.amazon.com/en/s3/). S3 es una solución de almacenamiento escalable y rentable que puede formar parte de sistemas de almacenamiento más abstractos como los data lakes y data warehousing. Los datos transformados se almacenan en un formato llamado Parquet que está optimizado para su uso analítico.

- **Capa de servicio:** [Amazon Athena](https://aws.amazon.com/en/athena/), un servicio de consultas, se utiliza para consultar los datos almacenados en un bucket de S3. Permite realizar consultas similares a SQL sin necesidad de extraer los datos de S3 y cargarlos en una base de datos tradicional. Los usuarios (en este caso el analista de datos) pueden ejecutar consultas ad-hoc con fines analíticos e informes.


<div id='4'/>

## 4 - Extrayendo, Transformando y Cargando Datos


Ahora que tienes una comprensión de los diferentes componentes de tu arquitectura de datos, primero configurarás el canal de datos y luego ejecutarás las diferentes etapas del mismo. Necesitas declarar y configurar los diferentes componentes de tu canal de datos. Para hacerlo, utilizarás [Terraform](https://www.terraform.io/), que es una de las herramientas de Infraestructura como Código (IaC) más populares desarrollada por HashiCorp. Permite a los usuarios definir y aprovisionar infraestructura utilizando un lenguaje de configuración declarativo (declarativo significa que los usuarios solo necesitan describir los componentes del canal de datos sin instruir sobre los pasos detallados necesarios para construir el canal de datos). Terraform es compatible con proveedores de servicios en la nube y también puede gestionar infraestructura local.


4.1. Verifique el contenido del directorio *terraform*. En ese directorio, encontrará algunos archivos con la extensión `tf`. Esos archivos definen las configuraciones de los recursos necesarios para su arquitectura de datos. Los recursos están separados en diferentes scripts solo por legibilidad, lo que significa que no necesariamente encontrará la misma estructura de archivos en todos los proyectos que utilizan Terraform para IaC. Intente tener una visión general de los recursos utilizados, pero no se preocupe si no entiende el código y todos los componentes en esta etapa.


4.2. Los archivos `tf` que verificaste te permiten declarar los recursos de tu arquitectura de datos. Aún necesitas especificar para AWS Glue cómo realizar la extracción, transformación y carga de datos. Puedes verificar todos estos pasos como un script de Python [terraform/assets/glue_job.py](terraform/assets/glue_job.py). Nuevamente, no necesitas preocuparte por los detalles. ¡Ahora pongamos en acción estos pasos!


4.3. Ejecuta el siguiente comando.

```bash
source scripts/setup.sh
```

Los archivos de configuración de Terraform dependen de algunas variables de entrada que representan el nombre del proyecto, los parámetros de la base de datos y otros nombres de recursos. El script `setup.sh` contiene las definiciones de estas variables. Una vez que lo ejecutes, obtendrá los valores de estas variables y los registrará como variables de entorno con el prefijo `TF_VAR_` en el archivo `~/.bashrc`. Luego, se ejecutará el script `~/.bashrc` para hacer que las variables estén disponibles para la configuración de Terraform.


4.4. En la terminal, navega hasta el directorio de terraform ejecutando el comando:

``` bash
cd terraform
```

*Nota*: Si hay errores en los comandos o archivos de configuración de Terraform, la terminal puede colapsar. 
Cuando esto suceda, verás el siguiente mensaje:

![etl_diagram](images/terminal_crash.png)

Puedes reabrir la terminal presionando <code>Ctrl + \`</code> (o <code>Cmd + \`</code>) o navegando a Ver > Terminal. 
En la terminal, vuelve a la carpeta de Terraform (`cd terraform`) y luego intenta 
ejecutar nuevamente los comandos requeridos. El error debería aparecer ahora en la terminal.
Si la terminal sigue colapsando, ejecuta en su lugar el siguiente comando:
`terraform apply -no-color  2> errors.txt`
Esto creará un archivo de texto que contiene el mensaje de error sin hacer que la terminal colapse.


4.5. Inicializa la configuración de Terraform con los proveedores requeridos:

``` bash
terraform init
```

Deberías obtener una salida similar a esta:

![terraform_init](./images/terraform_init.png)


4.6. Generar el plan de Terraform que te permite previsualizar los cambios en la infraestructura antes de aplicarlos. Cuando ejecutas el comando correspondiente, Terraform analiza los archivos de configuración para verificar el estado deseado, compara el estado deseado con el estado actual de la infraestructura y calcula las acciones necesarias para lograrlo. Ejecuta el siguiente comando para generar el plan:


``` bash
terraform plan
```

Obtendrás un resumen de los cambios planificados que indican los recursos que se agregarán, modificarán o eliminarán. Si falla al ejecutarse, verifica que hayas asignado correctamente las variables de entrada en el paso anterior.


4.7. Ir a la pestaña previamente abierta de la consola de AWS en tu navegador, buscar *Glue* y abrir *AWS Glue*. Ingresa *trabajos ETL* en el lado izquierdo del panel de control. Deberías ver que aún no hay trabajos definidos.


4.8. Aplica la configuración ejecutando:

``` bash
terraform apply
```

Te mostrará el mismo plan generado anteriormente y te pedirá que respondas `yes`:

![terraform_plan](images/terraform_plan.png)


4.9. Verifique el progreso de la implementación a través de la terminal.
Terraform creará algunos recursos en su cuenta de AWS. Una vez que termine, recibirá una confirmación similar a esta:

![stack_deployed](images/stack_deployed.png)


4.10. Regresa al panel de control de AWS Glue e ingresa en el enlace de *trabajos ETL*.
Ahora verás un trabajo creado con el nombre `de-c1w2-etl-job` que acabas de crear con IaC.


4.11. Iniciar el trabajo de AWS Glue:

``` bash
aws glue start-job-run --job-name de-c1w2-etl-job | jq -r '.JobRunId'
```

Deberías obtener `JobRunID` en la salida.


4.12. Verifique el estado del trabajo de AWS Glue intercambiando `<JobRunID>` con la salida del paso anterior:

``` bash
aws glue get-job-run --job-name de-c1w2-etl-job --run-id <JobRunID> --output text --query "JobRun.JobRunState"
```

También puede ver el estado del trabajo en la consola abriendo el `de-c1w2-etl-job` y yendo a la pestaña `Runs`. Espere hasta que el estado del trabajo cambie a `SUCCEEDED` (esto tomará de 2 a 3 minutos).


El trabajo de Glue transformó los datos, pasándolos de un 
[esquema original](https://www.mysqltutorial.org/mysql-sample-database.aspx) a otro:

![schema_after_ETL.png](images/schema_after_ETL.png)

Esta nueva forma de datos se conoce como esquema estrella. Consiste en tablas de hechos y dimensiones. La tabla fact_orders representa las mediciones que resultaron de un evento comercial. En este ejemplo, cada fila en la tabla de hechos corresponde a un pedido de venta y contiene mediciones relacionadas como la cantidad pedida y el precio. Las tablas de dimensiones proporcionan más contexto para las mediciones de la tabla de hechos. En este ejemplo, las tablas de dimensiones proporcionan más información relacionada con los clientes, las ubicaciones de los clientes y los detalles de los pedidos. ¿Cómo facilita esta forma el trabajo para los analistas de datos? La tabla de hechos contiene las mediciones que necesitan agregar (total de ventas, promedio de precios, ...) y las tablas de dimensiones ayudan a hacer estas agregaciones más específicas (total de ventas realizadas en un país dado, número máximo de cantidades pedidas para cada línea de productos). Esto se puede hacer escribiendo declaraciones de consulta simples, que podrían haber sido más complicadas si se hubieran realizado en la tabla original. Aprenderá más sobre modelado de datos en el Curso 4 de esta especialización.


<div id='5'/>

## 5 - Datos e información visual

Tu rol como Ingeniero de Datos podría terminar aquí. Todo lo que necesitas hacer ahora es darle acceso al analista de datos a los datos almacenados en S3 para que puedan consultarlos utilizando Amazon Athena. Para este laboratorio, también experimentarás con este último paso que es consultar los datos utilizando Amazon Athena y crear algunas visualizaciones para analizar los datos. Llevarás a cabo el análisis en Jupyter Notebook.


5.1. Abre el cuaderno *C1_W2_Dashboard.ipynb* y sigue las instrucciones allí.

Después de seguir las instrucciones del archivo del cuaderno, envía tu laboratorio haciendo clic en **Enviar tarea** (esquina superior derecha).
